In [1]:
from stop_words import get_stop_words
import os

In [23]:
random_seed = 42

In [2]:
stop_words = get_stop_words('ru')
for word in ['свой', 'р', 'г', 'назир', 'назиров', 'б']:
    stop_words.append(word)

In [9]:
documents = []

for d, dirs, files in os.walk('lemma_texts'):
    for f in files:
        text = open(os.path.join('.', d, f), 'r', encoding='utf-8').read()
        documents.append(text.replace('\n', ' ').replace('?', ''))

# NMF and LDA

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [12]:
# no_features = 1000

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [13]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [14]:
no_topics = 4
# diary, dostoevsky, literature, myth

# NMF
nmf = NMF(n_components=no_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

In [24]:
# LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)

/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [16]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()
    
no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
достоевский раскольников князь читатель писатель ставрогин преступление автор идея творчество
Topic 1:
миф сказка бог религия древний мифология обряд сюжет череп земля
Topic 2:
хрущев цк советский сша партия ссср москва октябрь фильм президент
Topic 3:
печорин онегин тургенев усадебный лермонтов сюжет пушкин обломов ленский усадьба

Topic 0:
достоевский советский фрейд война хрущев американский сша москва писатель октябрь
Topic 1:
достоевский миф бог сюжет князь идея любовь земля древний пушкин
Topic 2:
миф достоевский бог сюжет религия эпоха древний легенда сказка земля
Topic 3:
достоевский миф сюжет бог сказка любовь князь эпоха автор пушкин

